In [308]:
import OpenBlender
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [280]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [309]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [201]:
from functions import textTransform

In [4]:
os.chdir("..")

In [5]:
os.getcwd()

'/Users/yingyinxiao/Documents/Github/Stock-Market-Prediction/MSFT'

In [242]:
df = pd.read_csv("data/df_v2.csv").drop(columns = ["Unnamed: 0", "Unnamed: 0.1"], axis = 1)
df.head()

,date,volume,price,high,low,open,change,timestamp,FOXBUSINES.text_COUNT_last1days:microsoft,FOXBUSINES.text_last1days:microsoft,transformedDoc,FOXBUSINES.text_COUNT_last1days:negative,FOXBUSINES.text_last1days:negative,WALL_STREE.text_last1days:positive,WALL_STREE.text_COUNT_last1days:positive
0,02-01-2013 16:00:00,52.91M,27.62,27.73,27.15,27.25,3.41%,1.357142e+09,0,[],NaN,0,[],['the latest tweets and updates about the #fis...,6
1,03-01-2013 16:00:00,48.30M,27.25,27.65,27.16,27.63,-1.34%,1.357229e+09,2,['stocks ease after yesterdays big rally econo...,stocks ease after yesterdays big rally economy...,4,['amazons third-party sellers scored big this ...,['breaking: new jersey gov_ christie says hous...,3
2,04-01-2013 16:00:00,52.52M,26.74,27.34,26.73,27.27,-1.87%,1.357315e+09,2,['stocks mildly lower as traders mull mostly i...,stocks mildly lower as traders mull mostly inl...,2,['stocks mildly lower as traders mull mostly i...,['mark zuckerberg gave 498_8 million in facebo...,10
3,07-01-2013 16:00:00,37.12M,26.69,26.88,26.64,26.77,-0.19%,1.357574e+09,1,['stock futures edge lower after last weeks su...,stock futures edge lower after last weeks surg...,1,['stock futures edge lower after last weeks su...,"['buy a dvd or book, youve got something to pa...",5
4,08-01-2013 16:00:00,44.70M,26.55,26.79,26.46,26.75,-0.52%,1.357661e+09,0,[],NaN,0,[],"['@danahaggard ""assume positive intent"" seems ...",4


In [243]:
df.dtypes 
# change the type of "change" to float
# change the type of "date" to timestamp
# change the type of "volume" to float

date                                          object
volume                                        object
price                                        float64
high                                         float64
low                                          float64
open                                         float64
change                                        object
timestamp                                    float64
FOXBUSINES.text_COUNT_last1days:microsoft      int64
FOXBUSINES.text_last1days:microsoft           object
transformedDoc                                object
FOXBUSINES.text_COUNT_last1days:negative       int64
FOXBUSINES.text_last1days:negative            object
WALL_STREE.text_last1days:positive            object
WALL_STREE.text_COUNT_last1days:positive       int64
dtype: object

In [244]:
df['change'] = df['change'].apply(lambda x: eval(x[:-1]))
df['date'] = pd.to_datetime(df['date'])

In [245]:
for i in df.volume:
    if i[-1] != 'M':
        print(i[-1])

K


In [246]:
def changeVolumeType(v):
    if v[-1] == 'M':
        return eval(v[:-1])
    if v[-1] == 'K':
        return eval(v[:-1])*10**(-3)

In [247]:
df['volume'] = df['volume'].apply(changeVolumeType)

In [248]:
df['volumn(M)'] = df['volume']
df = df.drop('volume', axis = 1)
df.head()

,date,price,high,low,open,change,timestamp,FOXBUSINES.text_COUNT_last1days:microsoft,FOXBUSINES.text_last1days:microsoft,transformedDoc,FOXBUSINES.text_COUNT_last1days:negative,FOXBUSINES.text_last1days:negative,WALL_STREE.text_last1days:positive,WALL_STREE.text_COUNT_last1days:positive,volumn(M)
0,2013-02-01 16:00:00,27.62,27.73,27.15,27.25,3.41,1.357142e+09,0,[],NaN,0,[],['the latest tweets and updates about the #fis...,6,52.91
1,2013-03-01 16:00:00,27.25,27.65,27.16,27.63,-1.34,1.357229e+09,2,['stocks ease after yesterdays big rally econo...,stocks ease after yesterdays big rally economy...,4,['amazons third-party sellers scored big this ...,['breaking: new jersey gov_ christie says hous...,3,48.30
2,2013-04-01 16:00:00,26.74,27.34,26.73,27.27,-1.87,1.357315e+09,2,['stocks mildly lower as traders mull mostly i...,stocks mildly lower as traders mull mostly inl...,2,['stocks mildly lower as traders mull mostly i...,['mark zuckerberg gave 498_8 million in facebo...,10,52.52
3,2013-07-01 16:00:00,26.69,26.88,26.64,26.77,-0.19,1.357574e+09,1,['stock futures edge lower after last weeks su...,stock futures edge lower after last weeks surg...,1,['stock futures edge lower after last weeks su...,"['buy a dvd or book, youve got something to pa...",5,37.12
4,2013-08-01 16:00:00,26.55,26.79,26.46,26.75,-0.52,1.357661e+09,0,[],NaN,0,[],"['@danahaggard ""assume positive intent"" seems ...",4,44.70


In [296]:
# feature engineer the text 
def textTransform(doc, max_length = 13):
    exclude = set(string.punctuation)
    doc = eval(doc)
    sw = stopwords.words('english')
    porter = PorterStemmer()
    output = ''
    if (len(doc) == 0):
        return output
    else:
        for each in doc:
            each = re.sub('[0-9]+','',each) # remove numbers
            each = ''.join(ch for ch in each if ch not in exclude) # remove puntuations and extra spaces
            output += each
        output = [x for x in word_tokenize(output) if not len(x) > max_length]
        output_without_sw = [word for word in output if not word in sw]
        stemmed = [porter.stem(word) for word in output_without_sw] 
        return stemmed

In [297]:
df['transformedDoc'] = df['FOXBUSINES.text_last1days:microsoft'].apply(textTransform).apply(lambda x: " ".join(x) )

In [334]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english', ngram_range=(1,1))
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words= 'english', ngram_range=(1,1))

## Count vectorizer

In [335]:
count_wm = countvectorizer.fit_transform(df['transformedDoc'])
count_tokens = countvectorizer.get_feature_names()

In [336]:
prep_df = pd.DataFrame(count_wm.toarray(), columns=countvectorizer.get_feature_names())
prep_df.head()

,aal,aapl,aapldefens,aapltini,abandon,abrupt,absenc,absolut,acceler,access,...,yumbrand,yusuf,zigzag,zillow,zip,znga,zoe,zoom,zuckerberg,zynga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [337]:
df.columns

Index(['date', 'price', 'high', 'low', 'open', 'change', 'timestamp',
       'FOXBUSINES.text_COUNT_last1days:microsoft',
       'FOXBUSINES.text_last1days:microsoft', 'transformedDoc',
       'FOXBUSINES.text_COUNT_last1days:negative',
       'FOXBUSINES.text_last1days:negative',
       'WALL_STREE.text_last1days:positive',
       'WALL_STREE.text_COUNT_last1days:positive', 'volumn(M)'],
      dtype='object')

In [338]:
prep_df[['price', 'high', 'low', 'open', 'change','volumn(M)']] = df[['price', \
                                                                              'high', 'low', \
                                                                              'open', 'change','volumn(M)']]

In [339]:
# Where ‘change’ decreased more than 0.5%
prep_df['negative_poc'] = [1 if log_diff < -0.005 else 0 for log_diff in df['change']]
# Where ‘change’ increased more than 0.5%
prep_df['positive_poc'] = [1 if log_diff > 0.005 else 0 for log_diff in df['change']]

In [340]:
## autoregression that could be improved

In [341]:
# We need to shift our target one day so that we predict 
# for what will happen 'tomorrow' with news and data from 'today'
prep_df['target'] = prep_df['positive_poc'].shift(-1)
prep_df[['change', 'positive_poc', 'target']]

,change,positive_poc,target
0,3.41,1,0.0
1,-1.34,0,0.0
2,-1.87,0,0.0
3,-0.19,0,0.0
4,-0.52,0,1.0
...,...,...,...
1883,0.62,1,0.0
1884,-2.84,0,1.0
1885,1.51,1,0.0
1886,-4.96,0,0.0


In [342]:
prep_df = prep_df[:-1] # Remove the last one with NaN
# Define target
target = 'target'
# Create train/test sets
X = prep_df.loc[:, prep_df.columns != target].values
y = prep_df.loc[:,[target]].values
div = int(round(len(X) * 0.71))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
print('Train:')
print(X_train.shape)
print(y_train.shape)
print('Test:')
print(X_test.shape)
print(y_test.shape)

Train:
(1340, 3330)
(1340, 1)
Test:
(547, 3330)
(547, 1)


### RandomForestRegressor

In [344]:
%%time
rf = RandomForestRegressor(n_estimators = 1000, random_state = 1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


CPU times: user 1min 46s, sys: 903 ms, total: 1min 47s
Wall time: 1min 51s


In [345]:
print("AUC score:")
print(roc_auc_score(y_test, y_pred))
print('---')
# Let's binarize and look at the confusion matrix
preds = [1 if val > 0.5 else 0 for val in y_pred]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, preds))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test, preds))
print('---')

AUC score:
0.505611534365528
---
Confusion Matrix:
[[ 92 142]
 [114 199]]
---
Accuracy:
0.5319926873857403
---


### Stochastic gradient descent

In [347]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [348]:
print("AUC score:")
print(roc_auc_score(y_test, y_pred))
print('---')
# Let's binarize and look at the confusion matrix
preds = [1 if val > 0.5 else 0 for val in y_pred]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test, preds))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test, preds))
print('---')

AUC score:
0.5118033368832091
---
Confusion Matrix:
[[ 13 221]
 [ 10 303]]
---
Accuracy:
0.5776965265082267
---


## TF-IDF vectorizer

In [349]:
tfidf_wm = tfidfvectorizer.fit_transform(df['transformedDoc'])
tfidf_tokens = tfidfvectorizer.get_feature_names()

In [350]:
prep_df2 = pd.DataFrame(count_wm.toarray(), columns=tfidfvectorizer.get_feature_names())
prep_df2.head()

,aal,aapl,aapldefens,aapltini,abandon,abrupt,absenc,absolut,acceler,access,...,yumbrand,yusuf,zigzag,zillow,zip,znga,zoe,zoom,zuckerberg,zynga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [351]:
prep_df2[['price', 'high', 'low', 'open', 'change','volumn(M)']] = df[['price', \
                                                                              'high', 'low', \
                                                                              'open', 'change','volumn(M)']]

In [352]:
# Where ‘change’ decreased more than 0.5%
prep_df2['negative_poc'] = [1 if log_diff < -0.005 else 0 for log_diff in df['change']]
# Where ‘change’ increased more than 0.5%
prep_df2['positive_poc'] = [1 if log_diff > 0.005 else 0 for log_diff in df['change']]

In [353]:
# We need to shift our target one day so that we predict 
# for what will happen 'tomorrow' with news and data from 'today'
prep_df2['target'] = prep_df2['positive_poc'].shift(-1)
prep_df2[['change', 'positive_poc', 'target']]

,change,positive_poc,target
0,3.41,1,0.0
1,-1.34,0,0.0
2,-1.87,0,0.0
3,-0.19,0,0.0
4,-0.52,0,1.0
...,...,...,...
1883,0.62,1,0.0
1884,-2.84,0,1.0
1885,1.51,1,0.0
1886,-4.96,0,0.0


In [355]:
prep_df2 = prep_df2[:-1] # Remove the last one with NaN
# Define target
target = 'target'
# Create train/test sets
X2 = prep_df2.loc[:, prep_df2.columns != target].values
y2 = prep_df2.loc[:,[target]].values
div = int(round(len(X) * 0.71))
X_train2 = X2[:div]
y_train2 = y2[:div]
X_test2 = X2[div:]
y_test2 = y2[div:]
print('Train:')
print(X_train2.shape)
print(y_train2.shape)
print('Test:')
print(X_test2.shape)
print(y_test2.shape)

Train:
(1340, 3330)
(1340, 1)
Test:
(546, 3330)
(546, 1)


In [362]:
%%time
rf2 = RandomForestRegressor(n_estimators = 1000, random_state = 1)
rf2.fit(X_train2, y_train2)
y_pred2 = rf.predict(X_test2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


CPU times: user 1min 45s, sys: 745 ms, total: 1min 45s
Wall time: 1min 49s


In [363]:
print("AUC score:")
print(roc_auc_score(y_test2, y_pred2))
print('---')
# Let's binarize and look at the confusion matrix
preds2 = [1 if val > 0.5 else 0 for val in y_pred2]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test2, preds2))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test2, preds2))
print('---')

AUC score:
0.5068765511662028
---
Confusion Matrix:
[[ 92 141]
 [114 199]]
---
Accuracy:
0.532967032967033
---


In [359]:
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train2, y_train2)
y_pred2 = clf.predict(X_test2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [361]:
print("AUC score:")
print(roc_auc_score(y_test2, y_pred2))
print('---')
# Let's binarize and look at the confusion matrix
preds2 = [1 if val > 0.5 else 0 for val in y_pred2]
print('Confusion Matrix:')
print(metrics.confusion_matrix(y_test2, preds2))
print('---')
# Lets look at the accuracy score
print('Accuracy:')
print(accuracy_score(y_test2, preds2))
print('---')

AUC score:
0.5
---
Confusion Matrix:
[[  0 233]
 [  0 313]]
---
Accuracy:
0.5732600732600732
---


Tf-idf and count vectorizer have almost the same accuracy. We will improve the model by adding in more text. Further, we will consider using time series to improve the accuracy.